In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

In [2]:
NFOLDS = 3
SEED = 0
NROWS = None

In [3]:
data = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [4]:
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

In [5]:
gc.enable()

In [6]:
y_train = data['TARGET']
del data['TARGET']

In [7]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype =='object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

In [8]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [9]:
x_train = data.merge(right = avg_prev.reset_index(), how='left', on = 'SK_ID_CURR')
x_test = test.merge(right = avg_prev.reset_index(), how='left', on='SK_ID_CURR')

In [10]:
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [11]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [12]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

In [13]:
x_train = x_train[features]
x_test = x_test[features]

In [14]:
x_train.head()

,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,NAME_TYPE_SUITE_x,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,nb_app
0,0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,...,24.000000,3.000000,8.000000,365243.0,-565.000000,125.000000,-25.000000,-17.000000,0.000000,1.0
1,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,1,...,10.000000,1.000000,6.666667,365243.0,-1274.333333,-1004.333333,-1054.333333,-1047.333333,0.666667,3.0
2,1,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0,...,4.000000,0.000000,10.000000,365243.0,-784.000000,-694.000000,-724.000000,-714.000000,0.000000,1.0
3,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0,...,23.000000,2.888889,5.888889,365243.0,91066.500000,91584.000000,182477.500000,182481.750000,0.000000,9.0
4,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,...,20.666667,1.000000,3.333333,365243.0,-1263.200000,-837.200000,72136.200000,72143.800000,0.600000,6.0


In [15]:
kf = KFold(n_splits = NFOLDS, shuffle = True, random_state = SEED)

In [16]:
class SklearnWrapper(object):
    def __init__(self, clf, seed = 0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
    
class CatboostWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
    
    
class LightGBMWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
    
class XgbWrapper(object):
    def __init__(self, seed = 0, params = None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)
        
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label = y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)
    
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [17]:
# out of fold
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest)) # 3 X ntest
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]
        
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i,:] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis = 0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [18]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2    
}

In [19]:
xg = XgbWrapper(seed = SEED, params = xgb_params)
et = SklearnWrapper(clf = ExtraTreesClassifier, seed = SEED, params = et_params)
rf = SklearnWrapper(clf = RandomForestClassifier, seed = SEED, params = rf_params)
cb = CatboostWrapper(clf = CatBoostClassifier, seed = SEED, params = catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

[09:19:23] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:20:37] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:21:50] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting fl

In [20]:
print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))


XG-CV: 0.25966781672358985
ET-CV: 0.26296502347137407
RF-CV: 0.2629856213006646
RF-CV: 0.33097968739725225


In [21]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

(307511, 4),(48744, 4)


In [22]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train)

LogisticRegression()

In [23]:
test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]

In [24]:
test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')